# NN 2.0

## Defining data functions

Import needed libraries

In [1]:
# imports
import pandas as pd
import tensorflow as tf
import tensorflow.keras as ks
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

Get data from file and remove columns with text

In [2]:
def get_swissvotes_data(now:int=651, since:int=299)->pd.DataFrame:
    import re
    
    dataset = pd.read_csv("../data/formatted/swissvotes_edited_nov_2021.csv", sep=';')
    
    regex = re.compile("pdev_.*")
    to_excl = list(filter(regex.match, dataset.columns))
    
    dataset.drop(columns=to_excl, inplace=True)
    dataset.drop(columns=["legisjahr"], inplace=True)
    dataset.drop(columns=["titel_kurz_d", "titel_kurz_f", "titel_off_d", "titel_off_f", "stichwort"], inplace=True)
    dataset.drop(columns=["swissvoteslink", "anzahl", "anneepolitique", "bkchrono_de", "bkchrono_fr"], inplace=True)
    dataset.drop(columns=["curiavista_de", "curiavista_fr", "urheber", "bkresults_de", "bkresults_fr"], inplace=True)
    dataset.drop(columns=["bfsmap_de", "bfsmap_fr", "nach_cockpit_d", "nach_cockpit_f", "nach_cockpit_e"], inplace=True)
    dataset = dataset[dataset["anr"] < now] # we don't care about future votes
    dataset = dataset[dataset["anr"] >= since]
    
    return dataset
print(f"Defined {get_swissvotes_data}")

Defined <function get_swissvotes_data at 0x00000119EDA50DC0>


In [3]:
def get_left_right_data(now:int=651, since:int=299)->pd.DataFrame:
    col_names = ["anr", "datum", "titel_kurz_d", "ja_proz", "links_rechts", "kons_prog"]
    dataset = pd.read_csv("../data/formatted/brj_lire_konslib_edited.csv", sep=',')
    dataset = dataset[dataset["anr"].map(lambda x: x.replace(',', '.')).astype(float) < now]
    dataset = dataset.replace("#ZAHL!", np.nan).replace("Nan", np.nan)
    dataset = dataset[dataset["anr"].map(lambda x: x.replace(',', '.')).astype(float) >= since]
    
    return pd.DataFrame(dataset.drop(columns=["titel_kurz"]), index=get_swissvotes_data(now=now, since=since).index)

print(f"Defined {get_left_right_data}")

Defined <function get_left_right_data at 0x00000119EDA50D30>


In [4]:
def get_l_r_onehot(data:pd.DataFrame=get_left_right_data()["li_re"])->pd.DataFrame:
    temp = data.replace(np.nan, 4).astype(int)
    temp += 3
    tensor = tf.one_hot(temp, 8).numpy()
    
    temp = pd.DataFrame(tensor, columns=["l3", "l2", "l1", "nlr", "r1", "r2", "r3", "del"], index=data.index)
    return temp.drop(columns=["del"])

print(f"Defined {get_l_r_onehot}")

Defined <function get_l_r_onehot at 0x00000119EDA54550>


In [5]:
def get_l_r_exact(data:pd.DataFrame=get_left_right_data()["li_re"])->pd.DataFrame:
    temp = data.replace(np.nan, 0).astype(int)
    return temp + 3
print(f"Defined {get_l_r_exact}")

Defined <function get_l_r_exact at 0x00000119EDA8BF70>


In [6]:
def get_kons_prog_exact(data:pd.DataFrame=get_left_right_data()["kons_prog"])->pd.DataFrame:
    temp = data.replace(np.nan, 0).astype(int)
    return temp + 3
print(f"Defined {get_kons_prog_exact}")

Defined <function get_kons_prog_exact at 0x00000119EDA8B8B0>


In [7]:
def get_kons_prog_onehot(data:pd.DataFrame=get_left_right_data()["kons_prog"])->pd.DataFrame:
    temp = data.replace(np.nan, 4).astype(int)
    temp += 3
    tensor = tf.one_hot(temp, 8).numpy()
    
    temp = pd.DataFrame(tensor, columns=["k3", "k2", "k1", "nkp", "p1", "p2", "p3", "del"], index=data.index)
    return temp.drop(columns=["del"])

print(f"Defined {get_kons_prog_onehot}")

Defined <function get_kons_prog_onehot at 0x00000119EDA8B550>


In [8]:
def get_rechtsform_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["rechtsform"], 5).numpy();
    result = pd.DataFrame(tensor, columns=["ref_obl", "ref_fak", "initiative", "gegen_entw", "stichfr"], index=data.index)
    
    return result.astype(int)
print(f"Defined {get_rechtsform_onehot}")

Defined <function get_rechtsform_onehot at 0x00000119EDA54F70>


In [9]:
def get_politikbereich_multihot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    polber = data[["d1e1", "d2e1", "d3e1"]]
    polber = polber.replace('.', 0)
    polber = polber.astype(int)
    
    # the names of the columns (they're a bit long)
    cols = ["Staatsordnung", "Aussenpolitik", "Sicherheitspolitik", "Wirtschaft"]
    cols += ["Landwirtschaft", "Öffentliche Finanzen", "Energie", "Verkehr und Infrastruktur"]
    cols += ["Umwelt und Lebensraum", "Sozialpolitik", "Bildung und Forschung", "Kultur, Religion, Medien"]
    
    result = pd.DataFrame(columns=cols, index = data.index)
    for i in range(len(result)):
        row = np.zeros(len(cols))
        for p in polber.iloc[i]:
            if p != 0:
                row[p-1] = 1
        result.iloc[i] = row
    return result.astype(int)

print(f"Defined {get_politikbereich_multihot}")

Defined <function get_politikbereich_multihot at 0x00000119EEAC4D30>


In [10]:
def get_department_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    dep_single = data["dep"].replace('.', 2) # voting at age 18 is the only vote with a '.' and it's dep of inner
    dep_single = dep_single.astype(int)
    dep_names = ["EDA", "EDI", "EJPD", "VBS", "EFD", "WBF", "UVEK", "BK"]
    tensor = tf.one_hot(dep_single, len(dep_names)).numpy()
    result = pd.DataFrame(tensor, columns=dep_names, index=data.index)
    
    return result.astype(int)

print(f"Defined {get_department_onehot}")

Defined <function get_department_onehot at 0x00000119EDB43C10>


In [11]:
def get_bundesrat_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["br_pos"].replace('.', 3).astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Für_bund", "Dagegen_bund", "Keine_bund"], index=data.index)
    return result.astype(int)

print(f"Defined {get_bundesrat_onehot}")

Defined <function get_bundesrat_onehot at 0x00000119EEB56EE0>


In [12]:
def get_legislatur(low:int=0, high:int=3, data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    leg = data["legislatur"]
    def my_map(x:int, x_min:int=leg.min(0), x_max:int=leg.max(0), y_min:int=low, y_max:int=high)->float:
        return (x-x_min)/(x_max-x_min)*(y_max-y_min)+y_min
    
    normalized = data[["legislatur"]].applymap(my_map)
    return normalized

print(f"Defined {get_legislatur}")

Defined <function get_legislatur at 0x00000119EE2DA040>


In [13]:
def get_parlament_exact(name = "nr")->pd.DataFrame:
    data = get_swissvotes_data()[[name+"ja", name+"nein"]]
    ja = data[name+"ja"].replace('.', 0).astype(int)
    return ja/(ja+data[name+"nein"].replace('.', 1).astype(int))
print(f"Defined {get_parlament_exact}")

Defined <function get_parlament_exact at 0x00000119EE2DA790>


In [14]:
def get_parlament_onehot(data:pd.DataFrame = get_swissvotes_data()["nr_pos"])->pd.DataFrame:
    tensor = tf.one_hot(data.astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Für_"+data.name, "Dagegen_"+data.name, "Keine_"+data.name], index=data.index)
    return result.astype(int)

print(f"Defined {get_parlament_onehot}")

Defined <function get_parlament_onehot at 0x00000119EF92FF70>


In [15]:
def get_parties(data:pd.DataFrame = get_swissvotes_data())->list:
    import re
    
    regex_incl = re.compile("p_.*")
    regex_excl = re.compile("p_others_.*")
    
    parties_pre = list(filter(regex_incl.match, data.columns))
    parties = [p for p in parties_pre if not regex_excl.match(p)]
    return parties

print(f"Defined {get_parties}")

Defined <function get_parties at 0x00000119EE2DA670>


In [16]:
def normalize_party_reco(data:pd.DataFrame = get_swissvotes_data(), names:list = get_parties())->pd.DataFrame:
    # deal with unwanted values first
    normalized = data[names].replace(".", 0)
    normalized.replace(np.nan, 0, inplace=True)
    normalized = normalized.astype(int)
    normalized.replace([3,4,5,66,9999], 0, inplace=True)
    
    result = pd.DataFrame(index=normalized.index)
    
    for p in names: # go through parties and create one hot encoding
        tensor = tf.one_hot(normalized[p], 3).numpy()
        temp = pd.DataFrame(tensor, columns=[p+"_neutral", p+"_ja", p+"_nein"], index=result.index)
        result = result.join(temp)

    return result.astype(int)
print(f"Defined {normalize_party_reco}")

Defined <function normalize_party_reco at 0x00000119EE2E8EE0>


In [17]:
def get_vote_result(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    result = data["annahme"].replace('.', 0)
    return result.astype(int)

print(f"Defined {get_vote_result}")

Defined <function get_vote_result at 0x00000119EDE399D0>


In [18]:
# creates a dataframe 
def get_canton_results(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    import re
    regex = re.compile(".*_annahme")
    canton_names = list(filter(regex.match, data.columns))
    return data[canton_names].replace('.', 0).astype(int)

print(f"Defined {get_canton_results}")

Defined <function get_canton_results at 0x00000119EF559EE0>


In [19]:
def get_volk_proz(data:pd.DataFrame=get_swissvotes_data())->pd.DataFrame:
    temp = data["volkja_proz"]
    return temp / 100

print(f"Defined {get_volk_proz}")

Defined <function get_volk_proz at 0x00000119EEDCEEE0>


In [20]:
def get_canton_proz(data:pd.DataFrame=get_swissvotes_data(), name:str=None)->pd.DataFrame:
    if name != None:
        return data[name+"_japroz"].replace('.', 50).astype(float) / 100
    
    import re
    regex = re.compile(".._japroz")
    names = list(filter(regex.match, data.columns))
    return data[names].replace('.', 50).astype(float) / 100

print(f"Defined {get_canton_proz}")

Defined <function get_canton_proz at 0x00000119EEC66C10>


In [21]:
def get_canton_names()->list:
    import re
    regex = re.compile(".._japroz")
    return [x[0:2] for x in list(filter(regex.match, get_swissvotes_data().columns))]

print(f"Defined {get_canton_names}")

Defined <function get_canton_names at 0x00000119EEC66550>


## Training the net

In [22]:
# get data    
# the possible inputs for the neural net are:
    # Rechtsform (one hot),
    # Politikbereich (multi hot),
    # Department (one hot),
    # Position of the Bundesrat (one hot),
    # legislatur (normalized from 0-3),
    # Position of Nationalrat [0:1],
    # Position of Ständerat [0:1],
    # Party recommendations (one hot),
    # left, right rating (one hot),
    # conservative, liberal (one hot)
# the possible outputs are:
    # result of the votes (binary),
    # result on a canton level (binary)
    # yes percentage of the people ([0:1]),
    # yes percentage by canton ([0:1])
def get_data(input_func:list=[get_rechtsform_onehot,
                             get_politikbereich_multihot, 
                             get_department_onehot, 
                             get_bundesrat_onehot,
                             get_legislatur,
                             get_parlament_exact,
                             get_parlament_exact,
                             normalize_party_reco,
                             get_l_r_exact,
                             get_kons_prog_exact], 
            input_args:list=[None, 
                            None,
                            None,
                            None,
                            None,
                            None,
                            "sr",
                            None,
                            None,
                            None],
            output_func:list=[get_vote_result,
                             get_canton_results,
                             get_volk_proz, 
                             get_canton_proz],
            output_args:list=[None,
                             None,
                             None,
                             None]):
    input_list = []
    for f, a in zip(input_func, input_args):
        if type(a) == type(None):
            input_list.append(f())
        else:
            input_list.append(f(a))
    
    inputs = pd.concat(input_list, axis=1)
    
    out_list = []
    for f, a in zip(output_func, output_args):
        if type(a) == type(None):
            out_list.append(f())
        else:
            out_list.append(f(a))
    
    outputs = pd.concat(out_list, axis=1)
    
    return get_swissvotes_data(), inputs, outputs

print(f"Defined {get_data}")

Defined <function get_data at 0x00000119EEC660D0>


In [23]:
def create_model(name:str, input_size:int = len(get_data()[1].columns), hidden:list=[100, 50, 20],
                 output_size:int = len(get_data(output_func=[get_vote_result,get_canton_results], 
                                                output_args=[None,None])[2].columns), 
                 activation:str="relu", activation_output:str="sigmoid", 
                 optimizer=ks.optimizers.SGD(learning_rate=0.1), 
                 loss=ks.losses.BinaryCrossentropy(), 
                 metrics:list=[ks.metrics.BinaryAccuracy(), ks.metrics.FalseNegatives()])->ks.models.Sequential:
    model = ks.models.Sequential(name=name)
    
    model.add(ks.layers.Dense(units=input_size, activation=activation, name="Input"))
    
    for i in range(len(hidden)):
        model.add(ks.layers.Dense(units=hidden[i], activation=activation, name="Hidden_"+str(i)))
        model.add(ks.layers.Dropout(rate=.1, name="Dropout_"+str(i)))
        
    model.add(ks.layers.Dense(units=output_size, activation=activation_output, name="Output"))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

print(f"Defined {create_model}")

Defined <function create_model at 0x00000119EDE39280>


In [24]:
def train_model(model:ks.models.Sequential, 
                inputs:pd.DataFrame=get_data()[1], 
                outputs:pd.DataFrame=get_data(output_func=[get_vote_result,get_canton_results], 
                                              output_args=[None,None])[2], 
                test_size:float=0.6, batch_size:int=50, epochs:int=125, shuffle:bool=True, verbose=0)->tuple:
    
    from sklearn.model_selection import train_test_split as tss
    in_train, in_test, out_train, out_test = tss(inputs, outputs, test_size=test_size, shuffle=shuffle)
    
    history = model.fit(x=in_train, y=out_train, batch_size=batch_size, epochs=epochs, 
                        shuffle=shuffle, verbose=verbose)
    
    return history, in_test, out_test

print(f"Defined {train_model}")

Defined <function train_model at 0x00000119EEDCED30>


In [25]:
def predict_single(model:ks.models.Sequential, anr:int):
    data = get_data()
    inputs = data[1]
    inputs = inputs[data[0]["anr"] == anr]
    tensor = tf.convert_to_tensor(inputs)
    
    return model.predict(tensor)[0][0]
print(f"Defined {predict_single}")

Defined <function predict_single at 0x00000119F196D820>


### Create and Train models

There are two national models. One for binary results and one for exact percentage of yes votes.

In [26]:
optimizer = ks.optimizers.RMSprop(learning_rate=0.01)
epochs = 200

nat_bin_shuf = create_model("BinaryModel", optimizer=optimizer, output_size=1, 
                            metrics=[ks.metrics.MeanAbsoluteError()])
nat_bin_nons = create_model("BinaryModel", optimizer=optimizer, output_size=1, 
                            metrics=[ks.metrics.MeanAbsoluteError()])
nat_ext_shuf = create_model("ExactModel", optimizer=optimizer, output_size=1, loss=ks.losses.MeanAbsoluteError(), 
                           metrics=[ks.metrics.MeanSquaredError(), ks.metrics.MeanAbsoluteError()])
nat_ext_nons = create_model("ExactModel", optimizer=optimizer, output_size=1, loss=ks.losses.MeanAbsoluteError(), 
                           metrics=[ks.metrics.MeanSquaredError(), ks.metrics.MeanAbsoluteError()])

history_bin_shuf, in_test_bin_shuf, out_test_bin_shuf = train_model(nat_bin_shuf, epochs=epochs,
                                                     outputs=get_data(output_func=[get_vote_result], 
                                                                      output_args=[None])[2])
history_per_shuf, in_test_per_shuf, out_test_per_shuf = train_model(nat_ext_shuf, epochs=epochs,
                                                     outputs=get_data(output_func=[get_volk_proz], 
                                                                      output_args=[None])[2])
history_bin_nons, in_test_bin_nons, out_test_bin_nons = train_model(nat_bin_nons, epochs=epochs,
                                                     outputs=get_data(output_func=[get_vote_result], 
                                                                      output_args=[None])[2], shuffle=False)
history_per_nons, in_test_per_nons, out_test_per_nons = train_model(nat_ext_nons, epochs=epochs,
                                                     outputs=get_data(output_func=[get_volk_proz], 
                                                                      output_args=[None])[2], shuffle=False)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

In [27]:
res_bin_shuf = nat_bin_shuf.evaluate(x=in_test_bin_shuf, y=out_test_bin_shuf)
res_per_shuf = nat_ext_shuf.evaluate(x=in_test_per_shuf, y=out_test_per_shuf)
res_bin_nons = nat_bin_nons.evaluate(x=in_test_bin_nons, y=out_test_bin_nons)
res_per_nons = nat_ext_nons.evaluate(x=in_test_per_nons, y=out_test_per_nons)

from sklearn.metrics import r2_score

print(f"""Shuffeled:
\tBinary model results:
    \t\t{res_bin_shuf}
    \t\tR2: {r2_score(nat_bin_shuf.predict(in_test_bin_shuf), out_test_bin_shuf)}
\tExact model results:
    \t\t{res_per_shuf}
    \t\tR2:{r2_score(nat_ext_shuf.predict(in_test_per_shuf), out_test_per_shuf)}
Non-shuffeled:
\tBinary model results:
    \t\t{res_bin_nons}
    \t\tR2:{r2_score(nat_bin_nons.predict(in_test_bin_nons), out_test_bin_nons)}
\tExact model results:
    \t\t{res_per_nons}
    \t\tR2:{r2_score(nat_ext_nons.predict(in_test_per_nons), out_test_per_nons)}""")

7/7 [==============================] - 0s 429us/step - loss: 0.1073 - mean_squared_error: 0.0178 - mean_absolute_error: 0.1073
Shuffeled:
	Binary model results:
    		[1.158428430557251, 0.2277919203042984]
    		R2: 0.0634038079645145
	Exact model results:
    		[0.11128278821706772, 0.018461430445313454, 0.11128278821706772]
    		R2:-0.10125197084118875
Non-shuffeled:
	Binary model results:
    		[10.890706062316895, 0.3040306568145752]
    		R2:-1.489438097891688
	Exact model results:
    		[0.10732939839363098, 0.01784035749733448, 0.10732939839363098]
    		R2:0.15795234805014113


**Let's try to create a model for each canton.**

In [28]:
def get_per_canton_models(names:list=get_canton_names(), optimizer=ks.optimizers.SGD(learning_rate=0.01), 
                          loss=ks.losses.MeanAbsoluteError(), 
                          metrics:list=[ks.metrics.MeanSquaredError(), ks.metrics.MeanAbsoluteError()])->list:
    result = []
    for c in names:
        result.append((create_model("Canton_"+c, output_size=1, 
                                    optimizer=optimizer, loss=loss, 
                                    metrics=metrics), c))
    
    return result
print(f"Defined {get_per_canton_models}")

Defined <function get_per_canton_models at 0x00000119F57739D0>


In [29]:
def train_per_canton_models(train_inputs, train_outputs, 
                            models:list=np.array(get_per_canton_models())[:,0],  
                            test_size:float=0.6, batch_size:int=50, epochs:int=125, shuffle:bool=True)->list:
    result = []
    for t in models:
        result.append(t.fit(x=train_inputs, y=train_outputs, batch_size=batch_size, epochs=epochs, 
                               shuffle=shuffle, verbose=0))
        
    return result
        
print(f"Defined {train_per_canton_models}")

Defined <function train_per_canton_models at 0x00000119F556DC10>


In [30]:
def get_population_distribution():
    return {"zh":17.9, "be":12, "vd":9.4, "ag":8, "sg":5.9, "ge":5.8, "lu":4.8, "ti":4, 
                  "vs":4, "fr":3.8, "bl":3.4, "tg":3.3, "so": 3.2, "gr":2.3, "bs":2.3, 
                  "ne":2, "sz":1.9, "zg":1.5, "sh":1, "ju":0.9, "ar":0.6, "nw":0.5, "gl":0.5, 
                  "ow":0.4, "ur":0.4, "ai":0.2}

In [31]:
def predict_national_from_cantonal_outcome(model_tuples:list, inputs):
    population = get_population_distribution()
    s = model_tuples[0][0].predict(inputs) * 0
    for t in model_tuples:
        s += t[0].predict(inputs) * population[t[1]]
        
    return s/100
print(f"Defined {predict_national_from_cantonal_outcome}")

Defined <function predict_national_from_cantonal_outcome at 0x00000119F639B670>


In [32]:
def get_test_data_per_canton_models(test_size:float=0.2, shuffle:bool=True):
    data = get_data(output_func=[get_canton_proz, get_volk_proz], output_args=[None, None])
    
    from sklearn.model_selection import train_test_split as tss
    intr, inte, outr, oute = tss(data[1], data[2], test_size=test_size, shuffle=shuffle)
    outr = outr.drop(columns=["volkja_proz"])
    
    model_data = get_per_canton_models()
    models = [md[0] for md in model_data]
    train_per_canton_models(intr, outr, models, shuffle=shuffle)
    
    y_pred = np.squeeze(predict_national_from_cantonal_outcome(model_data, inte))
        
    return y_pred, oute["volkja_proz"]
print(f"Defined {get_test_data_per_canton_models}")

Defined <function get_test_data_per_canton_models at 0x00000119F639B820>


In [ ]:
y_pred_nonshuf, y_real_nonshuf = get_test_data_per_canton_models()
y_pred_shuffle, y_real_shuffle = get_test_data_per_canton_models(shuffle=False)
mse_nonshuf = ks.metrics.mean_squared_error(y_pred_nonshuf, y_real_nonshuf)
mae_nonshuf = ks.metrics.mean_absolute_error(y_pred_nonshuf, y_real_nonshuf)
mse_shuffle = ks.metrics.mean_squared_error(y_pred_shuffle, y_real_shuffle)
mae_shuffle = ks.metrics.mean_absolute_error(y_pred_shuffle, y_real_shuffle)
from sklearn.metrics import r2_score
r2_shuf = r2_score(y_pred_shuffle, y_real_shuffle)
r2_nons = r2_score(y_pred_nonshuf, y_real_nonshuf)
print(f"""National prediction from cantonal models:
        shuffled:
            mse:\t{mse_shuffle}
            mae:\t{mae_shuffle}
            r2:\t\t{r2_shuf}
        non shuffled:
            mse:\t{mse_nonshuf}
            mae:\t{mae_nonshuf}
            r2:\t\t{r2_nons}""")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
len(get_data()[1].columns)